In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pickle
import logging
import multiprocessing
from pathlib import Path

import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

import arviz as az
import numpyro

from hbmep.config import Config
from hbmep.model.utils import Site as site

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)

In [2]:
import numpyro.distributions as dist
from hbmep.model import Baseline

class SaturatedReLU(Baseline):
    LINK = "saturated_relu"

    def __init__(self, config: Config):
        super(SaturatedReLU, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]

    def _model(self, subject, features, intensity, response_obs=None):
        subject, n_subject = subject
        features, n_features = features
        intensity, n_data = intensity

        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)
        n_feature0 = n_features[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            global_sigma_b = numpyro.sample("global_sigma_b", dist.HalfNormal(100))
            global_sigma_v = numpyro.sample("global_sigma_v", dist.HalfNormal(100))

            global_sigma_L = numpyro.sample("global_sigma_L", dist.HalfNormal(1))

            global_sigma_g_1 = numpyro.sample("global_sigma_g_1", dist.HalfNormal(100))
            global_sigma_g_2 = numpyro.sample("global_sigma_g_2", dist.HalfNormal(100))

            global_sigma_p = numpyro.sample("global_sigma_p", dist.HalfNormal(100))

            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(site.mu_a, dist.HalfNormal(scale=5))
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(scale=1))

                sigma_b_raw = numpyro.sample("sigma_b_raw", dist.HalfNormal(scale=1))
                sigma_b = numpyro.deterministic(site.sigma_b, global_sigma_b * sigma_b_raw)

                sigma_v_raw = numpyro.sample("sigma_v_raw", dist.HalfNormal(scale=1))
                sigma_v = numpyro.deterministic(site.sigma_v, global_sigma_v * sigma_v_raw)

                sigma_L_raw = numpyro.sample("sigma_L_raw", dist.HalfNormal(scale=1))
                sigma_L = numpyro.deterministic(site.sigma_L, global_sigma_L * sigma_L_raw)

                sigma_g_1_raw = numpyro.sample("sigma_g_1_raw", dist.HalfNormal(scale=1))
                sigma_g_1 = numpyro.deterministic("sigma_g_1", global_sigma_g_1 * sigma_g_1_raw)

                sigma_g_2_raw = numpyro.sample("sigma_g_2_raw", dist.HalfNormal(scale=1))
                sigma_g_2 = numpyro.deterministic("sigma_g_2", global_sigma_g_2 * sigma_g_2_raw)

                sigma_p_raw = numpyro.sample("sigma_p_raw", dist.HalfNormal(scale=1))
                sigma_p = numpyro.deterministic("sigma_p", global_sigma_p * sigma_p_raw)

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    """ Priors """
                    a_raw = numpyro.sample("a_raw", dist.Gamma(concentration=mu_a, rate=1))
                    a = numpyro.deterministic(site.a, (1 / sigma_a) * a_raw)

                    b_raw = numpyro.sample("b_raw", dist.HalfNormal(scale=1))
                    b = numpyro.deterministic(site.b, sigma_b * b_raw)

                    v_raw = numpyro.sample("v_raw", dist.HalfNormal(scale=1))
                    v = numpyro.deterministic(site.v, sigma_v * v_raw)

                    L_raw = numpyro.sample("L_raw", dist.HalfNormal(scale=1))
                    L = numpyro.deterministic(site.L, sigma_L * L_raw)

                    g_1_raw = numpyro.sample("g_1_raw", dist.HalfCauchy(scale=1))
                    g_1 = numpyro.deterministic(site.g_1, sigma_g_1 * g_1_raw)

                    g_2_raw = numpyro.sample("g_2_raw", dist.HalfCauchy(scale=1))
                    g_2 = numpyro.deterministic(site.g_2, sigma_g_2 * g_2_raw)

                    p_raw = numpyro.sample("p_raw", dist.HalfNormal(scale=1))
                    p = numpyro.deterministic("p", sigma_p * p_raw)

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.data, n_data, dim=-2):
                """ Model """
                mu = numpyro.deterministic(
                    site.mu,
                    L[subject, feature0] - \
                    jnp.log(jnp.maximum(
                        1 / jnp.exp(v[subject, feature0]),
                        jnp.exp(-jax.nn.relu(
                            b[subject, feature0] * (intensity - a[subject, feature0])
                        ))
                    ))
                )
                beta = numpyro.deterministic(
                    site.beta,
                    g_1[subject, feature0] + g_2[subject, feature0] * jnp.power(1 / (mu + 1), p[subject, feature0])
                )

                """ Observation """
                numpyro.sample(
                    site.obs,
                    dist.Gamma(concentration=mu * beta, rate=beta),
                    obs=response_obs
                )


In [11]:
1 / jnp.exp(posterior_samples[site.v].mean(axis=0))

Array([[[0.48599227, 0.66196399, 0.73452507, 0.89136143, 0.90261237,
         0.97198998],
        [0.56851894, 0.61505362, 0.99782213, 0.92259522, 0.98544738,
         0.99868208]],

       [[0.69623027, 0.55432058, 0.69691807, 0.87493749, 0.90552971,
         0.98368356],
        [0.63711704, 0.57095507, 0.94487037, 0.93521493, 0.99965554,
         0.99736354]],

       [[0.74865138, 0.64033694, 0.80441455, 0.80341203, 0.93252947,
         0.9642759 ],
        [0.49007109, 0.55772332, 0.92376836, 0.80593411, 0.78755064,
         0.99212855]],

       [[0.53062564, 0.5090759 , 0.73238517, 0.9332914 , 0.89051803,
         0.97191822],
        [0.98254786, 0.78877355, 0.95090845, 0.9923865 , 0.97187579,
         0.9979659 ]],

       [[0.60685221, 0.65602473, 0.83762301, 0.91334217, 0.94688788,
         0.96860666],
        [0.47656949, 0.25667944, 0.95898661, 0.95988877, 0.98358896,
         0.99601797]],

       [[0.69224326, 0.58733639, 0.71256843, 0.90588174, 0.95385079,
         0.

In [3]:
toml_path = os.path.join("/home/vishu/repos/hbmep-paper/configs/paper/tms/link-comparison/saturated_relu.toml")

config = Config(toml_path=toml_path)
# config.MCMC_PARAMS["num_warmup"] = 10000
# config.MCMC_PARAMS["num_samples"] = 12000
# config.MCMC_PARAMS["thinning"] = 4

model = SaturatedReLU(config=config)

# from numpyro.infer import NUTS
# sampler = NUTS(model=model._model, max_tree_depth=50)


2023-10-27 02:01:17,684 - hbmep.config - INFO - Verifying configuration ...
2023-10-27 02:01:17,685 - hbmep.config - INFO - Success!
2023-10-27 02:01:17,700 - hbmep.model.baseline - INFO - Initialized model with saturated_relu link


In [4]:
src = "/home/vishu/data/hbmep-processed/human/tms/data_pkpk_auc.csv"
df = pd.read_csv(src)

# subset = ["SCA01"]
# ind = df[model.subject].isin(subset)
# df = df[ind].reset_index(drop=True).copy()

df, encoder_dict = model.load(df=df)


2023-10-27 02:01:23,766 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/saturated_relu
2023-10-27 02:01:23,766 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/saturated_relu
2023-10-27 02:01:23,768 - hbmep.dataset.core - INFO - Processing data ...
2023-10-27 02:01:23,769 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [5]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-10-27 02:01:25,394 - hbmep.model.baseline - INFO - Running inference with saturated_relu ...


  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

2023-10-27 02:52:46,657 - hbmep.utils.utils - INFO - func:run_inference took: 51 min and 21.26 sec


In [6]:
prediction_df = model.make_prediction_dataset(df=df)
posterior_predictive = model.predict(df=prediction_df, posterior_samples=posterior_samples)

model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples, prediction_df=prediction_df, posterior_predictive=posterior_predictive)
model.render_predictive_check(df=df, encoder_dict=encoder_dict, prediction_df=prediction_df, posterior_predictive=posterior_predictive)


2023-10-27 02:52:46,827 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.01 sec
2023-10-27 02:52:54,884 - hbmep.utils.utils - INFO - func:predict took: 8.06 sec
2023-10-27 02:52:55,047 - hbmep.model.baseline - INFO - Rendering ...
2023-10-27 02:53:05,002 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/saturated_relu/recruitment_curves.pdf
2023-10-27 02:53:05,005 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 10.12 sec
2023-10-27 02:53:05,205 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...
2023-10-27 02:53:23,645 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/saturated_relu/posterior_predictive_check.pdf
2023-10-27 02:53:23,646 - hbmep.utils.utils - INFO - func:_render_predictive_check took: 18.64 sec
2023-10-27 02:53:23,647 - hbmep.utils.utils - INFO - func:render_predictive_check took: 18.64 sec


In [7]:
mcmc.print_summary(prob=.95)



                         mean       std    median      2.5%     97.5%     n_eff     r_hat
       L_raw[0,0,0]      0.89      0.91      0.57      0.02      2.58      2.06      6.84
       L_raw[0,0,1]      1.00      0.71      0.97      0.09      2.15      2.19      4.17
       L_raw[0,0,2]      0.70      0.21      0.70      0.34      1.07      2.35      3.00
       L_raw[0,0,3]      0.85      0.47      0.74      0.17      1.75      3.81      1.84
       L_raw[0,0,4]      1.43      0.43      1.41      0.74      2.22      2.22      6.54
       L_raw[0,0,5]      0.49      0.27      0.59      0.03      0.76      2.03     11.17
       L_raw[0,1,0]      0.45      0.17      0.44      0.21      0.76      2.18      7.80
       L_raw[0,1,1]      0.48      0.15      0.46      0.32      0.72      2.02     14.63
       L_raw[0,1,2]      0.59      0.21      0.69      0.21      0.76      2.01     15.63
       L_raw[0,1,3]      0.21      0.08      0.19      0.12      0.36      2.05      7.91
       L_

In [8]:
numpyro_data = az.from_numpyro(mcmc)

""" Model evaluation """
logger.info("Evaluating model ...")

score = az.loo(numpyro_data)
logger.info(f"ELPD LOO (Log): {score.elpd_loo:.2f}")

score = az.waic(numpyro_data)
logger.info(f"ELPD WAIC (Log): {score.elpd_waic:.2f}")



2023-10-27 03:00:49,059 - __main__ - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1037: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
2023-10-27 03:01:00,011 - __main__ - INFO - ELPD LOO (Log): 7248.18
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities

In [11]:
import pickle

dest = os.path.join(model.build_dir, "inference.pkl")
with open(dest, "wb") as f:
    pickle.dump((model, mcmc, posterior_samples), f)


In [12]:
dest = os.path.join(model.build_dir, "numpyro_data.nc")
az.to_netcdf(numpyro_data, dest)


'/home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/rectified_logistic/numpyro_data.nc'